In [25]:
import pandas as pd

df = pd.read_csv('./final_named_and_final_cleaned_data_0_5_with_fram.csv')

df.head()

,Unnamed: 0,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,htn,...,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,framingham_score
0,0,63,1,1.0,1.0,1.0,3.0,1.0,145.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,18
1,1,67,1,1.0,1.0,1.0,3.0,4.0,160.0,1.0,...,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20
2,2,67,1,1.0,1.0,1.0,3.0,4.0,120.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,17
3,3,37,1,1.0,1.0,1.0,3.0,3.0,130.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7
4,4,41,0,1.0,0.0,1.0,1.0,2.0,130.0,1.0,...,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,9


In [26]:
X = df.drop('num', axis=1)
y = df['num']


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(max_iter=10000)

param_grid = {
    'C': [0.01, 0.1, 1, 10],  # Hiperparametr regularyzacji
    'solver': ['liblinear', 'saga'],  # Rozwiązywacze
    'penalty': ['l2', 'none'],  # Typy regularyzacji
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(f"Najlepsze parametry: {grid_search.best_params_}")
print(f"Dokładność na zbiorze testowym: {grid_search.best_estimator_.score(X_test, y_test)}")



Najlepsze parametry: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Dokładność na zbiorze testowym: 0.6148148148148148


C:\Users\emiar\Desktop\MOJE\STUDIA\eksploracja danych\Nowy folder\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
22 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\emiar\Desktop\MOJE\STUDIA\eksploracja danych\Nowy folder\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\emiar\Desktop\MOJE\STUDIA\eksploracja danych\Nowy folder\.venv\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "C:\Users\emiar\Desktop\MOJE\STUDIA\ek

In [29]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, grid_search.best_estimator_.predict(X_test)))

print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))


[[98  1  1  3  0]
 [31 25  6  6  0]
 [ 4 16 12 19  0]
 [ 1  6  3 26  0]
 [ 5  1  1  0  5]]
              precision    recall  f1-score   support

           0       0.71      0.95      0.81       103
           1       0.51      0.37      0.43        68
           2       0.52      0.24      0.32        51
           3       0.48      0.72      0.58        36
           4       1.00      0.42      0.59        12

    accuracy                           0.61       270
   macro avg       0.64      0.54      0.55       270
weighted avg       0.60      0.61      0.58       270



In [30]:
from sklearn.metrics import classification_report
import csv

def save_model_results(model_name, model, X_test, y_test, filename='model_results.csv'):
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred, output_dict=True)

    results = {
        'Model': model_name,
        'Accuracy': report['accuracy'],
        'Precision Class 0': report['0']['precision'],
        'Recall Class 0': report['0']['recall'],
        'F1-score Class 0': report['0']['f1-score'],
        'Precision Class 1': report['1']['precision'],
        'Recall Class 1': report['1']['recall'],
        'F1-score Class 1': report['1']['f1-score'],
        'Precision Class 2': report['2']['precision'],
        'Recall Class 2': report['2']['recall'],
        'F1-score Class 2': report['2']['f1-score'],
        'Precision Class 3': report['3']['precision'],
        'Recall Class 3': report['3']['recall'],
        'F1-score Class 3': report['3']['f1-score'],
        'Precision Class 4': report['4']['precision'],
        'Recall Class 4': report['4']['recall'],
        'F1-score Class 4': report['4']['f1-score']
    }

    file_exists = False
    try:
        with open(filename, 'r'):
            file_exists = True
    except FileNotFoundError:
        pass

    with open(filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=results.keys())

        if not file_exists:
            writer.writeheader()

        writer.writerow(results)




In [31]:
model.fit(X_train, y_train)
#save_model_results('Logistic Regression', model, X_test, y_test)

C:\Users\emiar\Desktop\MOJE\STUDIA\eksploracja danych\Nowy folder\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
